In [ ]:
! pip install  git+https://github.com/huggingface/transformers.git datasets accelerate scikit-learn -Uqq
!pip install -U transformers>=4.48.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 113.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets transformers torch

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_scheduler
from datasets import load_dataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Load the dataset
from datasets import load_dataset

imdb = load_dataset("imdb")

imdb["test"][0]


{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
model = AutoModelForMaskedLM.from_pretrained("answerdotai/ModernBERT-base")

def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",  # Pad to a fixed maximum length
        truncation=True,       # Truncate sequences exceeding max_length
        max_length=512,        # Set the maximum length (adjust as needed)
        return_tensors="pt"
    )

# Apply tokenization to the test set
tokenized_train_dataset = imdb["train"].map(tokenize_function, batched=True)
tokenized_test_dataset = imdb["test"].map(tokenize_function, batched=True)



In [ ]:
##!pip install --upgrade transformers

# Import necessary libraries
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoConfig
from torch.utils.data import DataLoader

config = AutoConfig.from_pretrained("answerdotai/ModernBERT-base")
model = AutoModelForSequenceClassification.from_config(config)

# Prepare DataLoaders
train_dataset = tokenized_train_dataset.remove_columns(['text']).rename_column('label', 'labels')
test_dataset = tokenized_test_dataset.remove_columns(['text']).rename_column('label', 'labels')

import numpy as np
from sklearn.metrics import f1_score

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    score = f1_score(
            labels, predictions, labels=labels, pos_label=1, average="weighted"
        )
    return {"f1": float(score) if score == 1 else score}


train_bsz, val_bsz = 32, 32
lr = 8e-5
betas = (0.9, 0.98)
n_epochs = 2
eps = 1e-6
wd = 8e-6
training_args = TrainingArguments(
    output_dir=f"fine_tuned_modern_bert",
    learning_rate=lr,
    per_device_train_batch_size=train_bsz,
    per_device_eval_batch_size=val_bsz,
    num_train_epochs=n_epochs,
    lr_scheduler_type="linear",
    optim="adamw_torch",
    adam_beta1=betas[0],
    adam_beta2=betas[1],
    adam_epsilon=eps,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=True,
    bf16_full_eval=True,
    push_to_hub=False,
)
# Create a Trainer instance
trainer = Trainer(
    model=model,                         # The pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Tokenized training dataset
    eval_dataset=test_dataset,           # Tokenized test dataset
    compute_metrics = compute_metrics    # Evaluation metrics
)

# Train the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

# Save the trained model and tokenizer
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")


Epoch,Training Loss,Validation Loss,F1
1,0.407300,0.371728,0.841483
2,0.181800,0.335045,0.874186


Evaluation Results: {'eval_loss': 0.33498910069465637, 'eval_f1': 0.8741856493132063, 'eval_runtime': 64.3186, 'eval_samples_per_second': 388.69, 'eval_steps_per_second': 12.158, 'epoch': 2.0}


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/tokenizer.json')

In [ ]:
# Example input text
new_texts = ["This movie is boring", "Spectacular"]

# Tokenize the input
inputs = tokenizer(new_texts, padding=True, truncation=True, return_tensors="pt")

# Move inputs to the same device as the model
inputs = inputs.to(model.device)
# Put the model in evaluation mode
model.eval()

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

print("Predictions:", predictions.tolist())